In [0]:
from dotenv import load_dotenv
import os
from typing import Literal, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt.tool_node import ToolNode, tools_condition



from databricks_langchain import (
    ChatDatabricks,
    VectorSearchRetrieverTool
)



# Initialize LLM
llm = ChatDatabricks(endpoint="databricks-gpt-oss-120b")



def call_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}


# Initialize the retriever tool.
vs_tool = VectorSearchRetrieverTool(
  index_name="agents.main.foodly_policy_embedding_index",
  tool_name="foodly_policy_document_retrieval_tool",
  num_results=2,
  tool_description="Use this tool to search the Foodly knowledge base for policies, procedures, and service-related information. It retrieves the most relevant chunks from the company’s official documentation, including refund rules, cancellation terms, delivery guidelines, loyalty program details, privacy policies, and escalation procedures"
)


llm_with_tools = llm.bind_tools([vs_tool])

builder = StateGraph(MessagesState)

builder.add_node("llm",call_llm)
builder.add_node("tools",ToolNode([vs_tool]))


builder.add_edge(START,"llm")
builder.add_conditional_edges("llm" , tools_condition)
builder.add_edge("tools","llm")


agent = builder.compile()



In [0]:
messages = agent.invoke({"messages": [HumanMessage("What are refund policies for Foodly?")]})

last_message = messages["messages"][-1].content
print(last_message)


**Foodly Refund Policy – at a glance**

| Situation | What you get | How it’s handled |
|-----------|--------------|------------------|
| **Order not delivered** (e.g., restaurant closed, no delivery partner available, extreme weather, system outage) | Full refund | Returned to the original payment method **or** issued instantly as Foodly credit. |
| **Incorrect or missing items** (wrong dish, missing side, wrong quantity) | Partial refund or credit equal to the value of the missing/incorrect items | Refund is processed once the issue is verified (usually within 24 h). |
| **Food‑quality problems** (spoiled, unsafe, contaminated) | Full refund **or** a free re‑delivery (if the restaurant can remake the order) | You can choose the preferred resolution; the refund/credit is applied immediately after approval. |
| **Duplicate payment** (accidental double charge) | Refund of the extra charge(s) | Processed automatically once the duplicate is detected or reported. |
| **Other eligible cases** (e.g., order cancelled by Foodly because a partner canceled) | Full refund | Same as above. |

### Non‑Refundable Cases  
Refunds are **not** issued for:

* An incorrect address supplied by the customer.  
* The customer being unreachable at the time of delivery.  
* Personal taste preferences (e.g., “too spicy” or “too bland”).  
* Delays caused by external factors (traffic, weather, local restrictions) **unless** the delivery is **> 90 minutes** later than the ETA.

### Refund Timelines  

| Refund type | Typical processing time |
|-------------|--------------------------|
| **Bank or card refunds** | 5–7 business days (depends on the card issuer). |
| **Foodly credits** | Instant – you see the credit in the app as soon as the refund is approved. |
| **Notification** | You receive an in‑app notification and an email confirming the refund. |

### How to Request a Refund  

1. **Open the order** in the Foodly app.  
2. Tap **“Report an Issue”** and select the appropriate reason (e.g., “Missing items,” “Food quality,” “Did not receive order,” etc.).  
3. Provide any relevant details or photos (especially for quality issues).  
4. Submit – Foodly’s support team will review the case, usually within a few hours, and will either:  
   * Approve a full/partial refund, or  
   * Offer a Foodly credit or re‑delivery.  

If you don’t see a resolution within 24 hours, you can follow up via the **Help Center** or contact **Live Chat** for escalation.

---

**Bottom line:** You’re eligible for a refund (or credit) when Foodly is at fault—undelivered orders, missing/incorrect items, unsafe food, or duplicate charges. Personal preference issues and address errors are excluded. Refunds to cards take a few business days; Foodly credits are immediate. If you need help, start the “Report an Issue” flow in the app.

In [0]:
messages = agent.invoke({"messages": [HumanMessage("What are refund policies for Foodly?")]})

last_message = messages["messages"][-1].content


import markdown

# Convert markdown → HTML
html = markdown.markdown(last_message, extensions=["tables", "fenced_code"] )

# Render in the notebook automatically
displayHTML(html)

In [0]:
messages = agent.invoke({"messages": [HumanMessage("I bought it like 10 days ago , can i make the refund?")]})

last_message = messages["messages"][-1].content
print(last_message)

In [0]:
messages = agent.invoke({"messages": [HumanMessage("check the order number 345555")]})

last_message = messages["messages"][-1].content
print(last_message)